In [1]:
#importar librerias necesarias
import pandas as pd   #crear dataframe
import requests       #para enviar la solicitud a la UR
from bs4 import BeautifulSoup #para obtener el contenido en forma de HTML
import numpy as np  # para contar los valores (en nuestro caso)

In [2]:
#asignar la URL con nombre de variable url
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
#request le permite enviar una solicitud HTTP
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
#creando una lista vacía, para que podamos agregar los valores
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
#Nota: estas tres listas se agregaron recientemente, no se explican en el video
description = []
Director = []
Stars = []

In [4]:
#almacenar los datos requeridos significativos en la variable
pelidatos = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})

In [5]:
#llamando uno por uno usando for loop
for store in pelidatos:
    name = store.h3.a.text
    movie_name.append(name)
    
    year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(year_of_release)
    
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
    
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
    
    meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
    metascore.append(meta)
    
    value = store.find_all('span', attrs = {'name': 'nv'})
    
    vote = value[0].text
    votes.append(vote)
    #since, bruto y votos tienen los mismos atributos, es por eso que creamos una variable común y luego usamos la indexación
    grosses = value[1].text if len(value) >1 else '***'
    gross.append(grosses)

In [6]:
# Descripción de las películas
describe = store.find_all('p', class_ = 'text-muted')
description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '***'
description.append(description_)

In [7]:
#Detalles del elenco -- Nombre del director de raspado y estrellas 
cast = store.find("p", class_ = '')
cast = cast.text.replace('\n', '').split('|')
cast = [x.strip() for x in cast]
cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
Director.append(cast[0])
Stars.append([x.strip() for x in cast[1].split(",")])

In [8]:
#creando un dataframe usando la biblioteca pandas
movie_DF = pd.DataFrame({'Pelicula': movie_name, 'Año': year, 'Tiempo': time, 'Rating': rating, 'Puntuación': metascore, 'Votos': votes, 'Recaudación': gross})

In [9]:
#Guardar datos en un archivo de Excel:
movie_DF.to_excel("Top_100_IMDB_Movies.xlsx")

In [10]:
movie_DF.head(7)

,Pelicula,Año,Tiempo,Rating,Puntuación,Votos,Recaudación
0,Sueños de fuga,1994,142,9.3,82,"2,754,396",$28.34M
1,El padrino,1972,175,9.2,100,"1,916,395",$134.97M
2,Spider-Man: A través del spider-verso,2023,140,9.0,86,"137,585",#11
3,Batman: El caballero de la noche,2008,152,9.0,84,"2,727,248",$534.86M
4,La lista de Schindler,1993,195,9.0,95,"1,388,611",$96.90M
5,12 hombres en pugna,1957,96,9.0,97,"815,943",$4.36M
6,El señor de los anillos: El retorno del rey,2003,201,9.0,94,"1,891,997",$377.85M


In [11]:
Peliculas_mas_vistas=movie_DF.groupby('Pelicula').max().Rating.sort_values(ascending=False).reset_index()

In [12]:
Peliculas_mas_vistas.head(10)

,Pelicula,Rating
0,Sueños de fuga,9.3
1,El padrino,9.2
2,12 hombres en pugna,9.0
3,La lista de Schindler,9.0
4,El señor de los anillos: El retorno del rey,9.0
5,Spider-Man: A través del spider-verso,9.0
6,Batman: El caballero de la noche,9.0
7,El padrino II,9.0
8,Tiempos violentos,8.9
9,"Lo bueno, lo malo y lo feo",8.8


In [13]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [14]:
import plotly.express as px
import cv2 

In [19]:
graf = movie_DF[['Pelicula', 'Rating' ]].dropna()

#Gráfica de Dispersión con las 03 columnas
fig = px.bar(graf, x= 'Pelicula', y='Rating', color ='Rating')
fig.show()

In [20]:
Mayor_tiempo_pelicula = movie_DF[['Pelicula','Tiempo']]
Mayor_tiempo_pelicula.head
Mayor_tiempo_pelicula.to_csv("nuevo archivo3.csv", index=False)

In [21]:
data = pd.read_csv('nuevo archivo3.csv') 
data_ordenada = data.sort_values(by='Tiempo', ascending=True)
top10_tiempo_pelicula = data_ordenada.tail(15)
top10_tiempo_pelicula

,Pelicula,Tiempo
18,Interestelar,169
24,Rescatando al soldado Ryan,169
92,Fuego contra fuego,170
99,Caracortada,170
82,3 Idiots,170
1,El padrino,175
10,El señor de los anillos: La comunidad del anillo,178
14,El señor de los anillos: Las dos torres,179
62,Avengers: Endgame,181
25,Milagros inesperados,189


In [22]:
fig = px.bar(top10_tiempo_pelicula, x = 'Pelicula', y = 'Tiempo', color = 'Pelicula',
            pattern_shape_sequence = ['x', '.', '+', '/', '.'])
fig.show()

![linea 1](https://user-images.githubusercontent.com/19308295/115926252-2b8a0c00-a448-11eb-9d9c-b43beaf0ff68.png)